In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, lit
from configparser import ConfigParser
import psycopg2
from IPython.display import display

In [3]:
# create a spark session
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Wake County Restaurants") \
    .getOrCreate()

# create a dataframe from the CSV
df = spark \
    .read \
    .format("csv") \
    .option("header", "true") \
    .load("./data/Restaurants_in_Wake_County_NC.csv")

df.show(5)

df.printSchema()
df.count()

+--------+-----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+--------------------+-----------------+--------+------------+-----------+-------------+
|OBJECTID|     HSISID|                NAME|            ADDRESS1|ADDRESS2|       CITY|STATE|POSTALCODE|   PHONENUMBER|  RESTAURANTOPENDATE|     FACILITYTYPE|PERMITID|           X|          Y|GEOCODESTATUS|
+--------+-----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+--------------------+-----------------+--------+------------+-----------+-------------+
|    1001|04092016024|                WABA|2502 1/2 HILLSBOR...|    null|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18T00:00:...|       Restaurant|    6952|-78.66818477|35.78783803|            M|
|    1002|04092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    null|       CARY|   NC|     27518|(919) 852-6651|2011-11-08T00:00:...|       Food Stand|    6953|-78.78211173|35

3440

In [4]:
# add, remap and drop columns
df = df.withColumn("county", lit("Wake")) \
    .withColumnRenamed("HSISID", "datasetId") \
    .withColumnRenamed("NAME", "name") \
    .withColumnRenamed("ADDRESS1", "address1") \
    .withColumnRenamed("ADDRESS2", "address2") \
    .withColumnRenamed("CITY", "city") \
    .withColumnRenamed("STATE", "state") \
    .withColumnRenamed("POSTALCODE", "zip") \
    .withColumnRenamed("PHONENUMBER", "tel") \
    .withColumnRenamed("RESTAURANTOPENDATE", "dateStart") \
    .withColumnRenamed("FACILITYTYPE", "type") \
    .withColumnRenamed("X", "geoX") \
    .withColumnRenamed("Y", "geoY") \
    .drop("OBJECTID") \
    .drop("PERMITID") \
    .drop("GEOCODESTATUS")

df.head()

Row(datasetId='04092016024', name='WABA', address1='2502 1/2 HILLSBOROUGH  ST ', address2=None, city='RALEIGH', state='NC', zip='27607', tel='(919) 833-1710', dateStart='2011-10-18T00:00:00.000Z', type='Restaurant', geoX='-78.66818477', geoY='35.78783803', county='Wake')

In [5]:
# create a unique id for each record
delim = lit("_")
df = df.withColumn("id", concat(df["state"], delim, df["county"], delim, df["datasetId"]))

df.head()
df.printSchema()

root
 |-- datasetId: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address1: string (nullable = true)
 |-- address2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- tel: string (nullable = true)
 |-- dateStart: string (nullable = true)
 |-- type: string (nullable = true)
 |-- geoX: string (nullable = true)
 |-- geoY: string (nullable = true)
 |-- county: string (nullable = false)
 |-- id: string (nullable = true)

